In [ ]:


from instrument import *
from tkinter import *
from consts import *
import sounddevice as sd
import numpy as np
import mido
import time 


colors = ["blue", "red", "yellow", "pink"]


class MidiSequencerTkPLUS:
    # análogo a lo anterior
    def __init__(self,tk,instrument=None):
        self.instruments = instrument

        print("ewhfdkjslñ")
        #si se pasa un vector vacio se settea a un instrumento
        if(len(instrument) == 0):
            self.instruments.append(Instrument(tk,amp=0.2,ratio=3,beta=0.6))
        c = 0
        j = 0
        # trato instrument com oun vector de instrumentos
        for i in instrument:
            self.instruments[j] = i
            j += 1

            frame = LabelFrame(tk, text="Midi Sequencer", bg="#908060")
            frame.pack(side=BOTTOM)
            frameFile = Frame(frame, highlightbackground=colors[c], highlightthickness=6)
            
            c += 1
            if(c == len(colors)): 
                c = 0
                
            frameFile.pack(side=TOP)
            Label(frameFile,text='Archivo MIDI: ').pack(side=LEFT)
    
            self.file = Entry(frameFile) #.pack(side=RIGHT)
            self.file.insert(14,"pirates.mid")
            self.file.pack(side=LEFT)

            self.transport = 0
            
            self.text = Text(frame,height=6,width=23)
            self.text.pack(side=RIGHT)
            playBut = Button(frame,text="Play", command=self.play)
            playBut.pack(side=TOP)
            #stopBut = Button(frame,text="Stop", command=self.stop)
            #stopBut.pack(side=BOTTOM)
            

        #########
        self.tick = 1
        self.state = 'off'
        
    # obtención de la secuencia midi (noteOn/Off) con tiempos relativos al inicio
    def getSeq(self,midiEvents):
        seq = []
        accTime = 0
        for m in midiEvents:
            accTime += m.time
            if m.type=='note_on':
                if m.velocity==0: seq.append((accTime,'noteOff',m.note+self.transport,m.channel))
                else: seq.append((accTime,'noteOn',m.note+self.transport,m.channel))    
            elif m.type=='note_off':
                seq.append((accTime,'noteOff',m.note+self.transport,m.channel))
        return seq

  
    def play(self):
        events = mido.MidiFile(self.file.get())
        seq = self.getSeq(events)
        print(seq)

        self.state = 'on'
        self.playLoop(seq)

    def playLoop(self,seq,item=0,accTime=0):   
        if item>=len(seq) or self.state =='off':
            return

        # ahora tenemos que procesar todos los ítems cuyo tiempo supere el crono accTime    
        while item<len(seq) and accTime>=seq[item][0]:
            (_,msg,midiNote,_chan) = seq[item]  # (time,'noteOff',midNote,channel)
            self.text.insert('6.0',  f'{msg} {midiNote}\n') 
            if msg=='noteOn':  
                self.instrument.noteOn(midiNote)                   
            else: # msg noteOff    
                self.instrument.noteOff(midiNote)                   
            item += 1 # y avanzmos ítem


        # avanzammos crono 
        accTime += self.tick/1000

        self.text.after(self.tick,lambda: self.playLoop(seq,item,accTime)) 

         
    def stop(self):
        self.instrument.stop()
        self.state = 'off'  


In [26]:
%load_ext autoreload
%autoreload 2

from midiSequencerTk import *
import os    
from instrument import *


def test():
    def callback(outdata, frames, time, status):    
        if status: print(status)    
        #print(inputs)
        s = np.sum([i.next() for i in inputs],axis=0)
        s = np.float32(s)
        outdata[:] = s.reshape(-1, 1)

    os.system('xset r off')
    tk = Tk()

    instrumentos = []
    instrumentos.append(Instrument(tk, "Holaaa", 0.3, 4, 0.5))
    #instrumentos.append(Instrument(tk, "Adiosss", 0.1, 3, 0.7))
    #instrumentos.append(Instrument(tk, "Huh"))

    seq = MidiSequencerTkPLUS(tk,instrumentos)
    #print(seq.seq)
    inputs = instrumentos

    stream = sd.OutputStream(samplerate=SRATE, channels=1, blocksize=CHUNK, callback=callback)
    stream.start()

    tk.mainloop()

    stream.close()
    os.system('xset r on')

test()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ewhfdkjslñ
ME CAGO
output underflow
output underflow
noteOn 48
noteOff 48
noteOn 50
noteOff 50
noteOn 52
noteOff 52
noteOn 53
noteOff 53
noteOn 48
noteOff 48
noteOn 50
noteOff 50
noteOn 52
noteOff 52
noteOn 48
noteOff 48
noteOn 50
noteOff 50
noteOn 48
noteOn 50
noteOff 48
noteOff 50
